In [1]:
import pandas as pd 
pd.options.mode.chained_assignment = None
import numpy as np 
from copy import deepcopy
from string import punctuation
from random import shuffle
from collections import Counter

import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence 

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

from nltk.tokenize import TweetTokenizer # a tweet tokenizer from nltk.
tokenizer = TweetTokenizer()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

C:\Users\saksh\anaconda3\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
def ingest():
    data = pd.read_csv('./cleaned_twitter_dataset.csv')
    print ('dataset loaded with shape', data.shape )   
    return data

data = ingest()
data.head(5)

dataset loaded with shape (13724, 15)


,Tweet_date,Tweet_time,Tweet_City,Tweet_Country,Tweet_account,Retweet_count,Tweet_Text,Created Date,tweet_without_stopwords,neg,neu,pos,vader_polarity,sentiment,text
0,4/1/2020,0:08:00,NaN,Australia,GSK_AU,0,ask award research excellence open nomination ...,2020-04-01 00:08:00,ask award research excellence open nomination ...,0.000,0.297,0.703,0.9349,positive,ask award research excellence open nomination ...
1,4/1/2020,0:35:00,NaN,Australia,GSK_AU,3,award research excellence open nomination awar...,2020-04-01 00:35:00,award research excellence open nomination awar...,0.000,0.419,0.581,0.9022,positive,award research excellence open nomination awar...
2,4/1/2020,0:45:00,Basel,Switzerland,Novartis News,31,face global crisis novartis mobilized ramp cap...,2020-04-01 00:45:00,face global crisis novartis mobilized ramp cap...,0.291,0.709,0.000,-0.6249,negative,face global crisis novartis mobilized ramp cap...
3,4/1/2020,0:46:00,Basel,Switzerland,Novartis News,2,information response visit,2020-04-01 00:46:00,information response visit,0.000,1.000,0.000,0.0000,neutral,information response visit
4,4/1/2020,0:59:00,Basel,Switzerland,Novartis News,4,clinical investigation assessing portfolio exi...,2020-04-01 00:59:00,clinical investigation assessing portfolio exi...,0.000,1.000,0.000,0.0000,neutral,clinical investigation assessing portfolio exi...


In [3]:
def tokenize(tweet):
    # print(tweet)
    try:
        tokens = tokenizer.tokenize(tweet)
        
        return tokens
    except:
        return 'NC'

In [4]:
def postprocess(data, n=100):
    # data = data.head(n)
    data['tokens'] = data['Tweet_Text'].progress_map(tokenize)  ## progress_map is a variant of the map function plus a progress bar. Handy to monitor DataFrame creations.
    # data = data[data.tokens != 'NC']
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data
tokenData = postprocess(data)

progress-bar: 100%|██████████| 13724/13724 [00:00<00:00, 41245.14it/s]


In [5]:
x_train, x_test = train_test_split(np.array(data.tokens),
                                                     test_size=0.2)

In [6]:
# Cosine similarity between sentences
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saksh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
data.tokens.size

13724

In [8]:
# Cosine Similarity


for ii in range(0,1000):
  for jj in range(ii+1,1000):
    X_list = data.tokens[ii]
    Y_list = data.tokens[jj]
    
    sw = stopwords.words('english') 

    X_set = {w for w in X_list if not w in sw}  
    Y_set = {w for w in Y_list if not w in sw} 

    l1 =[];l2 =[] 
    # form a set containing keywords of both strings  
    rvector = X_set.union(Y_set)  
    for w in rvector: 
        if w in X_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in Y_set: l2.append(1) 
        else: l2.append(0) 
    c = 0

    # cosine formula  
    for i in range(len(rvector)): 
            c+= l1[i]*l2[i] 
    cosine = c / float((sum(l1)*sum(l2))**0.5) 
    if cosine > 0.85 and cosine < 1.0:
        print("similarity: ", cosine) 
        print(data.Tweet_Text[ii])
        print(data.Tweet_Text[jj])
        print("**********************")

similarity:  0.8571428571428571
doug dm booking reference email address guide
david dm booking reference email address guide
**********************
similarity:  0.8571428571428571
doug dm booking reference email address guide
sunset dm booking reference email address guide
**********************
similarity:  0.9258200997725514
doug dm booking reference email address guide
dm booking reference email address guide
**********************
similarity:  0.9258200997725514
doug dm booking reference email address guide
dm booking reference email address guide
**********************
similarity:  0.8888888888888888
oscarspanna send direct message booking reference email address contact
lindsay send direct message booking reference email address contact
**********************
similarity:  0.8888888888888888
oscarspanna send direct message booking reference email address contact
send direct message booking reference email address contact number
**********************
similarity:  0.857142857142857

similarity:  0.8571428571428571
cisco customer experience portal coming learn watch
cisco customer experience portal coming watch svp customer
**********************
similarity:  0.8819171036881969
hear adam radford distinguished system engineer cisco cisco decnet critical
hear distinguished system engineer cisco cisco decnet critical
**********************
similarity:  0.8944271909999159
ot operational technology world extension campus learn latest development fr
operational technology world extension campus learn latest development
**********************
similarity:  0.9128709291752769
happy assist require detail account
lisa happy assist require detail account
**********************
similarity:  0.9128709291752769
happy assist require detail account
kirstynnwe happy assist require detail account
**********************
similarity:  0.9128709291752769
fill box bottom page user
ella fill box bottom page user
**********************
similarity:  0.9128709291752769
fill box bottom page us

similarity:  0.8571428571428571
dm booking reference email id mentioned booking check
hear dm booking reference email id mentioned
**********************
similarity:  0.8571428571428571
dm booking reference email id mentioned booking check
katie dm booking reference email id mentioned booking
**********************
similarity:  0.8660254037844387
replied back dm
back sooner replied back dm
**********************
similarity:  0.8571428571428571
send booking reference email address booking dm check
send dm booking reference full email address
**********************
similarity:  0.8571428571428571
send booking reference email address booking dm check
lakhvinder dm booking reference email address check
**********************
similarity:  0.8571428571428571
send booking reference email address booking dm check
share booking reference email address booking dm check
**********************
similarity:  0.8571428571428571
stuart dm booking reference email id mentioned booking
khalid dm booking 

similarity:  0.875
sofia feel free dm booking reference email address
charlotte feel free dm booking reference email address
**********************
similarity:  0.9258200997725514
sunset dm booking reference email address guide
dm booking reference email address guide
**********************
similarity:  0.9258200997725514
sunset dm booking reference email address guide
dm booking reference email address guide
**********************
similarity:  0.8660254037844387
need checking news announce
checking news announce
**********************
similarity:  0.8660254037844387
danish replied dm check
dm replied check into
**********************
similarity:  0.8944271909999159
shahs flight operating city information
flight operating city information flight
**********************
similarity:  0.8944271909999159
flight operating city information flight
tejashree flight operating city information
**********************
similarity:  0.8944271909999159
flight operating city information flight
vida fli

In [9]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i,v in tqdm(enumerate(tweets)):
        label = '%s_%s'%(label_type,i)
        labelized.append(LabeledSentence(v, [label]))
    return labelized

x_train = labelizeTweets(x_train, 'TRAIN')
x_test = labelizeTweets(x_test, 'TEST')

0it [00:00, ?it/s]C:\Users\saksh\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """
10979it [00:00, 118995.88it/s]
2745it [00:00, 268765.22it/s]


In [10]:
tweet_w2v = Word2Vec(size=2000, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm(x_train)])
tweet_w2v.train([x.words for x in tqdm(x_train)],total_examples=tweet_w2v.corpus_count,epochs=100)

100%|██████████| 10979/10979 [00:00<?, ?it/s]


(5188115, 7931700)

In [11]:
tweet_w2v.wv.most_similar('ventilator')

[('army', 0.577003538608551),
 ('glove', 0.4836082458496094),
 ('expected', 0.48352015018463135),
 ('germany', 0.4555148482322693),
 ('speaks', 0.42191076278686523),
 ('religious', 0.39863359928131104),
 ('source', 0.39807915687561035),
 ('machine', 0.39361804723739624),
 ('giggle', 0.39022988080978394),
 ('spain', 0.38599926233291626)]

In [12]:
tweet_w2v.wv.most_similar('corona')

[('giovanni', 0.5872619152069092),
 ('table', 0.5127657651901245),
 ('ke', 0.5046003460884094),
 ('shiva', 0.4788028299808502),
 ('agar', 0.46836555004119873),
 ('anna', 0.44233238697052),
 ('hotspot', 0.43924766778945923),
 ('pandemicnbut', 0.39524349570274353),
 ('lifenplease', 0.3948119878768921),
 ('positive', 0.3914201855659485)]

In [ ]:
from gensim.models import FastText
# print(x_train)
f2vec = FastText(size=100, window=5, min_count=5, workers=4,sg=1)
f2vec.build_vocab([x.words for x in tqdm(x_train)])
f2vec.train([x.words for x in tqdm(x_train)],total_examples=tweet_w2v.corpus_count,epochs=100)

100%|██████████| 10979/10979 [00:00<?, ?it/s]


In [ ]:
f2vec.wv.most_similar('cough')

In [ ]:
f2vec.wv.most_similar('corona')

In [ ]:
f2vec.wv.most_similar('ventilator')

In [ ]:
# SkipGram with Fast text
f2vec = FastText(size=100, window=5, min_count=5, workers=4,sg=0)
f2vec.build_vocab([x.words for x in tqdm(x_train)])
f2vec.train([x.words for x in tqdm(x_train)],total_examples=tweet_w2v.corpus_count,epochs=100)


In [ ]:
f2vec.wv.most_similar('ventilator')

In [ ]:
f2vec.wv.most_similar('pandemic')

In [ ]:

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE


# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors each is of 2000 dimensions
word_vectors = [tweet_w2v[w] for w in list(tweet_w2v.wv.vocab.keys())[:]]

# dimensionality reduction. converting the vectors to 2d vectors
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(tweet_w2v.wv.vocab.keys())[:]

In [ ]:

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)